In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib.ticker import PercentFormatter
from scipy import stats as st
from scipy.stats import norm, beta
# !pip install --no-input kaleido==0.2.1
import kaleido #0.2.1
# !pip uninstall --no-input plotly
# !pip install --no-input plotly==5.5.0
import plotly
import plotly.io as pio #plotly 5.5.0
import plotly.graph_objects as go
import plotly_express as px
from plotly.subplots import make_subplots
import gc
import asyncio
from async_timeout import timeout
#from asgiref.sync import sync_to_async
import time,datetime
import signal

In [ ]:
%run ./EDA1V_new.ipynb
;

## Continuous numeric features

In [ ]:
# Numeric variable .description() check
# 
# min/max vs quantiles, mean, and standard deviation – looking for outliers/sensible results
# 
# Distribution plot examination
# 
# can use .hist() command or a more customized subplot group or for-each loop to plot a seperate histogram of each variable

In [ ]:
#Numeric vars
numerical_var_series=dfXY[dfXY.columns[dfXY.dtypes == np.float64]].columns#.to_list()#[0:5]
print(type(numerical_var_series))
print(len(numerical_var_series))
numerical_var_series

In [ ]:
#Create the different groups of columns to plot
proceso_vars_series=dfXY[numerical_var_series[(~numerical_var_series.str.contains("AP1_Furnace",regex=False)) & (~numerical_var_series.str.contains("pyro2",regex=False))]].columns#.head(1)
zoneTemp_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Temp.*",regex=True)]].columns#.head(1)

zoneAirGas_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Air.*|.*Zone.*Gas.*",regex=True)]].columns#.head(1)

recuperator_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains("AP1_Furnace.*Dilution|Recuperator|RWP|Combustion.*",regex=True)]].columns#.head(1)

pyro_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*pyro.*|.*Pyro.*",regex=True)]].columns#.head(1)

rest_vars_series=dfXY[[var for var in numerical_var_series.to_list() if var not in proceso_vars_series.to_list() and var not in zoneTemp_vars_series.to_list() and var not in zoneAirGas_vars_series.to_list() and var not in recuperator_vars_series.to_list() and var not in pyro_vars_series.to_list()]].columns#.head(1)

In [ ]:
#Show groups of vars
#dfXY[dfXY.columns[dfXY.dtypes == np.float64]].hist(figsize=(20,10))#Too many vars: 175
print("There are {} numerical vars".format(dfXY[dfXY.columns[dfXY.dtypes == np.float64]].shape))
print("There are {} numerical vars of tbl proceso".format(len(proceso_vars_series)))
print("There are {} numerical vars of zone temp".format(len(zoneTemp_vars_series)))
print("There are {} numerical vars of zone air/gas flow".format(len(zoneAirGas_vars_series)))
print("There are {} numerical vars of recuperator".format(len(recuperator_vars_series)))
print("There are {} numerical vars of pyro".format(len(pyro_vars_series)))
print("The rest of numerical vars of furnace are {} ".format(len(rest_vars_series)))

if dfXY[dfXY.columns[dfXY.dtypes == np.float64]].shape[1]==len(proceso_vars_series)+len(zoneTemp_vars_series)+len(zoneAirGas_vars_series)+len(recuperator_vars_series)+len(pyro_vars_series)+len(rest_vars_series):
    print("All groups sum total length")

### Plot zone air/gas flow: 126 vars


In [ ]:
#Zone2 air SP are scaled with outdated scale, rescale:outdated scale: 47565 SCFH->updated scale: 59655 SCFH
dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone2.*AirFlowSet[Pp]oint.*",regex=True)]]=dfXY.loc[:,zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone2.*AirFlowSet[Pp]oint.*",regex=True)]]*47565/59655

##### $Create$ $Gas$ $SP$ $vars$:

In [ ]:
dfRatioSP=dfXY[zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*RatioSP.*|.*RatioSet[Pp]oint.*",regex=True)]]#.reset_index(drop=True)
dfRatioSP.columns = range(dfRatioSP.columns.size)
dfAirFlowSP=dfXY[zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*AirFlowSP.*|.*AirFlowSet[Pp]oint.*",regex=True)]]#.reset_index(drop=True)
dfAirFlowSP.columns = range(dfAirFlowSP.columns.size)
dfGasFlowSP = dfAirFlowSP/dfRatioSP
dfGasFlowSP.columns = ['AP1_FurnaceZone0BottomGasFlowSP', 'AP1_FurnaceZone0TopGasFlowSP', 'AP1_FurnaceZone1BottomGasFlowSP', 'AP1_FurnaceZone1TopGasFlowSP', 'AP1_FurnaceZone2BottomGasFlowSP', 'AP1_FurnaceZone2TopGasFlowSP', 'AP1_FurnaceZone3BottomGasFlowSP', 'AP1_FurnaceZone3TopGasFlowSP', 'AP1_FurnaceZone4BottomGasFlowSP', 'AP1_FurnaceZone4TopGasFlowSP', 'AP1_FurnaceZone5BottomGasFlowSP', 'AP1_FurnaceZone5TopGasFlowSP', 'AP1_FurnaceZone6BottomGasFlowSP', 'AP1_FurnaceZone6TopGasFlowSP', 'AP1_FurnaceZone7BottomGasFlowSP', 'AP1_FurnaceZone7TopGasFlowSP', 'AP1_FurnaceZone8BottomGasFlowSP', 'AP1_FurnaceZone8TopGasFlowSP']
dfGasFlowSP.describe()

##### $Create$ $Error$ $df$

In [ ]:
dfAirFlowFdbk=dfXY[zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone.*Air.*Feedback.*|.*Zone.*Air.*Fdbk.*",regex=True)]]
dfAirFlowFdbk.head()
dfAirFlowFdbk.columns = range(dfAirFlowFdbk.columns.size)

dfGasFlowFdbk=dfXY[zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*Zone.*Gas.*Feedback.*|.*Zone.*Gas.*Fdbk.*",regex=True)]]
dfGasFlowFdbk.head()
dfGasFlowFdbk.columns = range(dfGasFlowFdbk.columns.size)

dfAirFlowSP.head()

dfGasFlowSP.head()
dfGasFlowSP.columns = range(dfGasFlowSP.columns.size)

dfFlowsErr = pd.merge(left=dfGasFlowFdbk-dfGasFlowSP,right=dfAirFlowFdbk-dfAirFlowSP,left_index=True,right_index=True,how='inner')
dfFlowsErr.columns = ['AP1_FurnaceZone0BottomGasFlowErr', 'AP1_FurnaceZone0TopGasFlowErr', 'AP1_FurnaceZone1BottomGasFlowErr', 'AP1_FurnaceZone1TopGasFlowErr', 'AP1_FurnaceZone2BottomGasFlowErr', 'AP1_FurnaceZone2TopGasFlowErr', 'AP1_FurnaceZone3BottomGasFlowErr', 'AP1_FurnaceZone3TopGasFlowErr', 'AP1_FurnaceZone4BottomGasFlowErr', 'AP1_FurnaceZone4TopGasFlowErr', 'AP1_FurnaceZone5BottomGasFlowErr', 'AP1_FurnaceZone5TopGasFlowErr', 'AP1_FurnaceZone6BottomGasFlowErr', 'AP1_FurnaceZone6TopGasFlowErr', 'AP1_FurnaceZone7BottomGasFlowErr', 'AP1_FurnaceZone7TopGasFlowErr', 'AP1_FurnaceZone8BottomGasFlowErr', 'AP1_FurnaceZone8TopGasFlowErr','AP1_FurnaceZone0BottomAirFlowErr', 'AP1_FurnaceZone0TopAirFlowErr', 'AP1_FurnaceZone1BottomAirFlowErr', 'AP1_FurnaceZone1TopAirFlowErr', 'AP1_FurnaceZone2BottomAirFlowErr', 'AP1_FurnaceZone2TopAirFlowErr', 'AP1_FurnaceZone3BottomAirFlowErr', 'AP1_FurnaceZone3TopAirFlowErr', 'AP1_FurnaceZone4BottomAirFlowErr', 'AP1_FurnaceZone4TopAirFlowErr', 'AP1_FurnaceZone5BottomAirFlowErr', 'AP1_FurnaceZone5TopAirFlowErr', 'AP1_FurnaceZone6BottomAirFlowErr', 'AP1_FurnaceZone6TopAirFlowErr', 'AP1_FurnaceZone7BottomAirFlowErr', 'AP1_FurnaceZone7TopAirFlowErr', 'AP1_FurnaceZone8BottomAirFlowErr', 'AP1_FurnaceZone8TopAirFlowErr']
dfFlowsErr.head()
dfFlowsErr.describe()

###### $Scatter$ $Plots$

In [ ]:
#Define function plot scatters zones
def func_scatter_plot_zone(zoneXAirGas_vars_series:pd.Series,ZoneXfig:str,TopBottom: str):
    
    dfZoneXScatter = dfXY[zoneXAirGas_vars_series[zoneXAirGas_vars_series.str.contains(str(TopBottom),regex=False) & ~zoneXAirGas_vars_series.str.contains("Ratio",regex=False)].tolist()+["SteelFamCluster"]]
    var_list = dfZoneXScatter.columns.to_list()[:-1]
    
    fig,axes = plt.subplots(5,5,figsize=(30,35))
    fig.suptitle(f"ScatterPlot of numerical {ZoneXfig+TopBottom} Air/Gas Flow variables",fontsize=30,va='top',y=1)
    
    for r in range(len(var_list)):
        for c in range(len(var_list)):
            if r==c:
                sns.histplot(dfZoneXScatter,x=var_list[r],ax=axes[r,c],kde=True,color="fuchsia",hue='SteelFamCluster')#data=pd.merge(left=dfZone8Scatter,right=pd.DataFrame(SteelFamCluster,index=dfXY.index,columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)
                axes[r,c].set_title(label=var_list[r],loc='center',pad=10,fontsize='large')
                axes[r,c].set_xlabel(xlabel="",labelpad=0)
                axes[r,c].set_ylabel(ylabel="",labelpad=0)
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
            if r>c:
                sns.scatterplot(dfZoneXScatter,x=var_list[c],y=var_list[r],ax=axes[r,c],hue='SteelFamCluster')#,size=20
                sns.regplot(data=dfZoneXScatter,x=var_list[c],y=var_list[r],ax=axes[r,c],color='darkorange',line_kws={'lw':0.8,'color':'olive'},order=1,scatter=False)#,marker='x',scatter_kws={'s':20}
                axes[r,c].set_title(label='ScatterPlot',loc='center',pad=10,fontsize='x-large')
                axes[r,c].set_xlabel(xlabel=var_list[c],labelpad=5,fontsize='large')
                axes[r,c].set_ylabel(ylabel=var_list[r],labelpad=10,fontsize='large')
                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(dfZoneXScatter[var_list[c]],dfZoneXScatter[var_list[r]])[0]**2,st.pearsonr(dfZoneXScatter[var_list[c]],dfZoneXScatter[var_list[r]])[1])],fontsize='large')
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
            if r<c:
                x=dfZoneXScatter[var_list[c]].to_numpy()
                y= dfZoneXScatter[var_list[r]].to_numpy()
                y_pred = LinearRegression().fit(x.reshape(-1,1),y.reshape(-1,1)).predict(x.reshape(-1,1))
                y_pred=y_pred[:,0]
                data=pd.merge(left=pd.DataFrame({"x":x,"res":y-y_pred}),right=pd.DataFrame(dfXY[["SteelFamCluster"]].reset_index(drop=True),columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)#"x":np.linspace(0,100,num=len(y))
                sns.scatterplot(data=data,x='x',y='res',hue='SteelFamCluster',ax=axes[r,c],marker="o")#,size=20,style='clustersGMM'
                axes[r,c].axhline(c='g',lw=0.8)
                #sns.residplot(data=dfZone0Scatter,x=var_list[c],y=var_list[r],ax=axes[r,c],order=1,color='darkolivegreen',scatter_kws={'s':20,'marker':'x'})
                axes[r,c].set_title(label='Residuals',loc='center',pad=10,fontsize='x-large')
                axes[r,c].set_xlabel(xlabel=var_list[c],labelpad=5,fontsize='large')
                axes[r,c].set_ylabel(ylabel=var_list[r],labelpad=10,fontsize='large')
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
    
    #Make space between subplots
    fig.tight_layout(pad=3)
    
    plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\ScatterPlots\newAirGasScatterPlot{}.png".format(str(ZoneXfig)+str(TopBottom)))
    #plt.savefig("/doc/NB_dir/js5296/AP1/EDA1V/img/eda1v/AirGasScatterPlot{}.png".format(str(ZoneXfig)+str(TopBottom)))
    # #Print correlation matrix
    # fig2,axes2 = plt.subplots(figsize=(6,4))
    # fig2.suptitle("Correlation Matrix of numerical Zone0 Top Air/Gas Flow variables",fontsize=15,va='top',y=0.97)
    # sns.set_context('paper', font_scale=1.4)
    # dfZone0Scatter_mx = dfZone0Scatter.corr()
    # mask = np.zeros_like(dfZone0Scatter_mx, dtype=bool)
    # mask[np.triu_indices_from(mask)] = True
    # sns.heatmap(dfZone0Scatter_mx, annot=True,mask=mask,cmap='Greens',square=True,ax=axes2)#cmap = sns.diverging_palette(220, 10, as_cmap=True)

In [ ]:
#Define function plot downsample scatters zones
def func_downsample_scatter_plot_zone(zoneXAirGas_vars_series:pd.Series,ZoneXfig:str,TopBottom:str,DownSample:str="rnd"):
    dfZoneXScatter = dfXY[zoneXAirGas_vars_series[zoneXAirGas_vars_series.str.contains(str(TopBottom),regex=False) & ~zoneXAirGas_vars_series.str.contains("Ratio",regex=False)].tolist()+["SteelFamCluster"]]
    if DownSample=="rnd":
        dfZoneXScatter=dfZoneXScatter.sample(n=2000,replace=False,random_state=1,axis=0)
    else:#evenly downsample
        dfZoneXScatter=dfZoneXScatter.iloc[[int(x) for x in list(np.round(np.linspace(0,dfZoneXScatter.shape[0]-1,2000),0))],:]#Downsample evenly
    var_list = dfZoneXScatter.columns.to_list()[:-1]
    
    fig,axes = plt.subplots(5,5,figsize=(30,35))
    fig.suptitle(f"DownSample ScatterPlot of numerical {ZoneXfig+TopBottom} Air/Gas Flow variables",fontsize=30,va='top',y=1)
    
    for r in range(len(var_list)):
        for c in range(len(var_list)):
            if r==c:
                sns.histplot(dfZoneXScatter,x=var_list[r],ax=axes[r,c],kde=True,color="fuchsia",hue='SteelFamCluster')#data=pd.merge(left=dfZone8Scatter,right=pd.DataFrame(SteelFamCluster,index=dfXY.index,columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)
                axes[r,c].set_title(label=var_list[r],loc='center',pad=10,fontsize='large')
                axes[r,c].set_xlabel(xlabel="",labelpad=0)
                axes[r,c].set_ylabel(ylabel="",labelpad=0)
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
            if r>c:
                sns.scatterplot(dfZoneXScatter,x=var_list[c],y=var_list[r],ax=axes[r,c],hue='SteelFamCluster')#,size=20
                sns.regplot(data=dfZoneXScatter,x=var_list[c],y=var_list[r],ax=axes[r,c],color='darkorange',line_kws={'lw':0.8,'color':'olive'},order=1,scatter=False)#,marker='x',scatter_kws={'s':20}
                axes[r,c].set_title(label='ScatterPlot',loc='center',pad=10,fontsize='x-large')
                axes[r,c].set_xlabel(xlabel=var_list[c],labelpad=5,fontsize='large')
                axes[r,c].set_ylabel(ylabel=var_list[r],labelpad=10,fontsize='large')
                axes[r,c].legend(['r\u00b2={:.2f}, p={:.2E}'.format(st.pearsonr(dfZoneXScatter[var_list[c]],dfZoneXScatter[var_list[r]])[0]**2,st.pearsonr(dfZoneXScatter[var_list[c]],dfZoneXScatter[var_list[r]])[1])],fontsize='large')
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
            if r<c:
                x=dfZoneXScatter[var_list[c]].to_numpy()
                y= dfZoneXScatter[var_list[r]].to_numpy()
                y_pred = LinearRegression().fit(x.reshape(-1,1),y.reshape(-1,1)).predict(x.reshape(-1,1))
                y_pred=y_pred[:,0]
                data=pd.merge(left=pd.DataFrame({"x":x,"res":y-y_pred}),right=pd.DataFrame(dfXY[["SteelFamCluster"]].reset_index(drop=True),columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)#"x":np.linspace(0,100,num=len(y))
                sns.scatterplot(data=data,x='x',y='res',hue='SteelFamCluster',ax=axes[r,c],marker="o")#,size=20,style='clustersGMM'
                axes[r,c].axhline(c='g',lw=0.8)
                #sns.residplot(data=dfZone0Scatter,x=var_list[c],y=var_list[r],ax=axes[r,c],order=1,color='darkolivegreen',scatter_kws={'s':20,'marker':'x'})
                axes[r,c].set_title(label='Residuals',loc='center',pad=10,fontsize='x-large')
                axes[r,c].set_xlabel(xlabel=var_list[c],labelpad=5,fontsize='large')
                axes[r,c].set_ylabel(ylabel=var_list[r],labelpad=10,fontsize='large')
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
    
    #Make space between subplots
    fig.tight_layout(pad=3)
    
    plt.savefig(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\ScatterPlots\newDownSampleAirGasScatterPlot{}.png".format(str(ZoneXfig)+str(TopBottom)))
    #plt.savefig("/doc/NB_dir/js5296/AP1/EDA1V/img/eda1v/AirGasScatterPlot{}.png".format(str(ZoneXfig)+str(TopBottom)))
    # #Print correlation matrix
    # fig2,axes2 = plt.subplots(figsize=(6,4))
    # fig2.suptitle("Correlation Matrix of numerical Zone0 Top Air/Gas Flow variables",fontsize=15,va='top',y=0.97)
    # sns.set_context('paper', font_scale=1.4)
    # dfZone0Scatter_mx = dfZone0Scatter.corr()
    # mask = np.zeros_like(dfZone0Scatter_mx, dtype=bool)
    # mask[np.triu_indices_from(mask)] = True
    # sns.heatmap(dfZone0Scatter_mx, annot=True,mask=mask,cmap='Greens',square=True,ax=axes2)#cmap = sns.diverging_palette(220, 10, as_cmap=True)

zone0AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone0",regex=False)]
dfZoneXScatter = dfXY[zone0AirGas_vars_series[zone0AirGas_vars_series.str.contains(str("Top"),regex=False) & ~zone0AirGas_vars_series.str.contains("Ratio",regex=False)].tolist()+["SteelFamCluster"]]
var_list = dfZoneXScatter.columns.to_list()[:-1]
fig = px.scatter(dfZoneXScatter,x=var_list[0],y=var_list[1],trendline="ols",color="SteelFamCluster",trendline_scope="overall",render_mode="webgl",opacity=1)
fig.update_traces(marker=dict(line=dict(width=0.9)))
fig.layout
#fig.show()
#fig.write_image(r"C:\Users\js5296\Desktop\NAS\Projects\AP1 Furnace Control\EDAs\old\1V\Dash\trial.webp",engine="kaleido")
#fig.write_image("/doc/NB_dir/js5296/AP1/trial.webp",engine="kaleido")
#fig.write_image("trial.webp",engine="kaleido")
#pio.write_image(fig,"/doc/NB_dir/js5296/AP1/trial.webp",engine="kaleido")

In [ ]:
gasFdbk_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*GasFlowF.*db.*k.*",regex=True)]
gasFdbk_vars_series.name="gasFdbk_vars_series"
airFdbk_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*AirFlowF.*db.*k.*",regex=True)]
airFdbk_vars_series.name="airFdbk_vars_series"
airSP_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*AirFlow[Ss]{1}.*[Pp]{1}.*",regex=True)]
airSP_vars_series.name="airSP_vars_series"
gasCV_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*GasFlowC.*V.*",regex=True)]
gasCV_vars_series.name="gasCV_vars_series"
airCV_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains(".*AirFlowC.*V.*",regex=True)]
airCV_vars_series.name="airCV_vars_series"

In [ ]:
gasFdbk_vars_series

In [ ]:
#Define function plot scatters plotly
def func_scatter_plotly_AirGas(xlist_series:pd.Series,ylist_series:pd.Series):
    ncol=9
    nrow=2
    var=0
    subtitles_list=[]
    for i in range(2):#Bottom/Top
        for j in range(9):#9zones
            if i==0:
                subtitles_list.append(f"Zone{j} Top")
            else:
                subtitles_list.append(f"Zone{j} Bottom")
            
    fig = make_subplots(rows=nrow,cols=ncol,subplot_titles=subtitles_list,vertical_spacing=0.1)#vertical_spacing(0-1)

    for r in range(1,nrow+1):
        if r==1:
            var=1
        else:
            var=0
        for c in range(1,ncol+1):
            #Create x,y,yhat
            x=np.asarray(dfXY[xlist_series[var]]).reshape(-1,1)
            y= np.asarray(dfXY[ylist_series[var]])
            y_pred = LinearRegression().fit(x,y).predict(x)
            x=x[:,0]
            #plot Scatters
            if (r==1) & (c==1):
                fig.add_trace(go.Scatter(x=dfXY[dfXY.SteelFamCluster=="HighGoal"][xlist_series[var]], 
                        y=dfXY[dfXY.SteelFamCluster=="HighGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="HighGoal",
                         legendgroup="group1",
                         marker=dict(color="blue",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ),row=r,col=c)
                fig.add_trace(go.Scatter(x=dfXY[dfXY.SteelFamCluster=="LowGoal"][xlist_series[var]], 
                        y=dfXY[dfXY.SteelFamCluster=="LowGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="LowGoal",
                         legendgroup="group2",
                         marker=dict(color="green",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ),row=r,col=c)
            else:
                fig.add_trace(go.Scatter(x=dfXY[dfXY.SteelFamCluster=="HighGoal"][xlist_series[var]], 
                        y=dfXY[dfXY.SteelFamCluster=="HighGoal"][ylist_series[var]], 
                        mode='markers', 
                         #zorder=3,
                         name="HighGoal",
                         legendgroup="group1",
                         marker=dict(color="blue",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ,showlegend=False),row=r,col=c)
                fig.add_trace(go.Scatter(x=dfXY[dfXY.SteelFamCluster=="LowGoal"][xlist_series[var]], 
                        y=dfXY[dfXY.SteelFamCluster=="LowGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="LowGoal",
                         legendgroup="group2",
                         marker=dict(color="green",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ,showlegend=False),row=r,col=c)
    
            #plot regression line
            fig.add_trace(go.Scatter(x=x, 
                    y=y_pred, 
                    mode='lines', 
                    #zorder=4,
                    #name='2',
                    line=dict(width=1.5,color="orange"),
                    showlegend=False),row=r,col=c)
            #plot regression data
            fig.add_annotation(x=dfXY[xlist_series[var]].min(), y=dfXY[ylist_series[var]].max()+10,
                    #text="r\u00b2={:.2f}, p={:.2E}".format(st.pearsonr(x,y)[0]**2,st.pearsonr(x,y)[1]),
                   text="r\u00b2={:.2f}".format(st.pearsonr(x,y)[0]**2),
                   #hovertext="pepe",
                   align='center',
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   #clicktoshow="onout",
                   #captureevents=True,
                   #xref='x2',
                   #yref='y2',
                   row=r,
                   col=c,
                  bordercolor='black',
                  borderpad=4,
                   borderwidth=2,
                  bgcolor='white',
                  font=dict(color='black'))
            #Add xaxis labels
            #fig.update_xaxes(title_text=xlist_series[var], row=r, col=c)#range=[40, 80],showgrid=False,type="log",
            #Add yaxis labels
            #fig.update_yaxes(title_text=ylist_series[var], row=r, col=c)
            #update var to plot
            var+=2



            
    #Add title and size and show img
    fig.update_layout(height=900, width=1850,title_text=f'x={xlist_series.name} vs y={ylist_series.name} across all zones', showlegend=True,title_x=0.5)#height=500*9, width=1000, #, legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
    #fig.show()
    if not os.path.exists(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\Dash"):
        os.mkdir(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\Dash")
    fig.write_html("C:\\Users\\js5296\\Desktop\\NAS\\Projects\\AP1_Furnace_Control\\EDAs\\new\\1V\\Dash\\new_{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    #fig.write_image("C:\\Users\\js5296\\Desktop\\NAS\\Projects\\AP1_Furnace_Control\\EDAs\\old\\1V\\Dash\\{}_vs_{}.webp".format(ylist_series.name,xlist_series.name))
    # fig.write_html("/doc/NB_dir/js5296/AP1/EDA1V/img/{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    # fig.write_image("/doc/NB_dir/js5296/AP1/EDA1V/img/{}_vs_{}.webp".format(ylist_series.name,xlist_series.name))
    del fig
    gc.collect()
    #return fig.layout

func_scatter_plotly_AirGas(xlist_series=gasCV_vars_series,ylist_series=gasFdbk_vars_series)

In [ ]:
#Define function plot scatters plotly
async  def func_downsample_scatter_plotly_AirGas(xlist_series:pd.Series,ylist_series:pd.Series):#async 
    dfDown=dfXY.iloc[[int(x) for x in list(np.round(np.linspace(0,dfXY.shape[0]-1,2000),0))],:]#Downsample evenly
    ncol=9
    nrow=2
    var=0
    subtitles_list=[]
    for i in range(2):#Bottom/Top
        for j in range(9):#9zones
            if i==0:
                subtitles_list.append(f"Zone{j} Top")
            else:
                subtitles_list.append(f"Zone{j} Bottom")
            
    fig = make_subplots(rows=nrow,cols=ncol,subplot_titles=subtitles_list,vertical_spacing=0.1)#vertical_spacing(0-1)

    for r in range(1,nrow+1):
        if r==1:
            var=1
        else:
            var=0
        for c in range(1,ncol+1):
            #Create x,y,yhat
            x=np.asarray(dfDown[xlist_series[var]]).reshape(-1,1)
            y= np.asarray(dfDown[ylist_series[var]])
            y_pred = LinearRegression().fit(x,y).predict(x)
            x=x[:,0]
            #plot Scatters
            if (r==1) & (c==1):
                fig.add_trace(go.Scatter(x=dfDown[dfDown.SteelFamCluster=="HighGoal"][xlist_series[var]], 
                        y=dfDown[dfDown.SteelFamCluster=="HighGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="HighGoal",
                         legendgroup="group1",
                         marker=dict(color="blue",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ),row=r,col=c)
                fig.add_trace(go.Scatter(x=dfDown[dfDown.SteelFamCluster=="LowGoal"][xlist_series[var]], 
                        y=dfDown[dfDown.SteelFamCluster=="LowGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="LowGoal",
                         legendgroup="group2",
                         marker=dict(color="green",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ),row=r,col=c)
            else:
                fig.add_trace(go.Scatter(x=dfDown[dfDown.SteelFamCluster=="HighGoal"][xlist_series[var]], 
                        y=dfDown[dfDown.SteelFamCluster=="HighGoal"][ylist_series[var]], 
                        mode='markers', 
                         #zorder=3,
                         name="HighGoal",
                         legendgroup="group1",
                         marker=dict(color="blue",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ,showlegend=False),row=r,col=c)
                fig.add_trace(go.Scatter(x=dfDown[dfDown.SteelFamCluster=="LowGoal"][xlist_series[var]], 
                        y=dfDown[dfDown.SteelFamCluster=="LowGoal"][ylist_series[var]], 
                        mode='markers', 
                        #zorder=3,
                         name="LowGoal",
                         legendgroup="group2",
                         marker=dict(color="green",line_width=0.6)
                        #,marker=dict(color=dfZoneXScatter.SteelFamCluster.map({"HighGoal":"blue","LowGoal":"green"}),line_width=0.6)
                            ,showlegend=False),row=r,col=c)
    
            #plot regression line
            fig.add_trace(go.Scatter(x=x, 
                    y=y_pred, 
                    mode='lines', 
                    #zorder=4,
                    #name='2',
                    line=dict(width=1.5,color="orange"),
                    showlegend=False),row=r,col=c)
            #plot regression data
            fig.add_annotation(x=dfDown[xlist_series[var]].min(), y=dfDown[ylist_series[var]].max()+10,
                    #text="r\u00b2={:.2f}, p={:.2E}".format(st.pearsonr(x,y)[0]**2,st.pearsonr(x,y)[1]),
                   text="r\u00b2={:.2f}".format(st.pearsonr(x,y)[0]**2),
                   #hovertext="pepe",
                   align='center',
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   #clicktoshow="onout",
                   #captureevents=True,
                   #xref='x2',
                   #yref='y2',
                   row=r,
                   col=c,
                  bordercolor='black',
                  borderpad=4,
                   borderwidth=2,
                  bgcolor='white',
                  font=dict(color='black'))
            #Add xaxis labels
            #fig.update_xaxes(title_text=xlist_series[var], row=r, col=c)#range=[40, 80],showgrid=False,type="log",
            #Add yaxis labels
            #fig.update_yaxes(title_text=ylist_series[var], row=r, col=c)
            #update var to plot
            var+=2



            
    #Add title and size and show img
    fig.update_layout(height=900, width=1850,title_text=f'DownSample x={xlist_series.name} vs y={ylist_series.name} across all zones', showlegend=True,title_x=0.5)#height=500*9, width=1000, #, legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
    #fig.show()
    if not os.path.exists(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\Dash"):
        os.mkdir(r"C:\Users\js5296\Desktop\NAS\Projects\AP1_Furnace_Control\EDAs\new\1V\Dash")
    # fig.write_html("C:\\Users\\js5296\\Desktop\\NAS\\Projects\\AP1_Furnace_Control\\EDAs\\old\\1V\\Dash\\DownSample_{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    # fig.write_image("C:\\Users\\js5296\\Desktop\\NAS\\Projects\\AP1_Furnace_Control\\EDAs\\old\\1V\\Dash\\DownSample_{}_vs_{}.webp".format(ylist_series.name,xlist_series.name))
    print("html save")
    fig.write_html("DownSample_{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    fig.write_html("C:\\Users\\js5296\\Desktop\\NAS\\Projects\\AP1_Furnace_Control\\EDAs\\new\\1V\\Dash\\newDownSample_{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    #fig.write_html("/doc/NB_dir/js5296/AP1/EDA1V/img/DownSample_{}_vs_{}.html".format(ylist_series.name,xlist_series.name))
    # print("webp save")#GIVES ERROR IN LOCAL ENV
    # fig.write_image("{}_vs_{}.png".format(ylist_series.name,xlist_series.name))
    # fig.write_image("{}_vs_{}.jpeg".format(ylist_series.name,xlist_series.name))
    # fig.write_image("{}_vs_{}.webp".format(ylist_series.name,xlist_series.name))
    # fig.write_image("/doc/NB_dir/js5296/AP1/EDA1V/img/DownSample_{}_vs_{}.webp".format(ylist_series.name,xlist_series.name))#For coldmill jupyter
    
    del fig
    gc.collect()
    #return fig.layout

#Only for UNIX
def timeout_handler(num, stack):#function that python process uses when SIGALRM is send
    print("Received SIGALRM")
    raise Exception("FUBAR")
    
signal.signal(signal.SIGALRM, timeout_handler)#give funct to use when SIGALRM send
signal.alarm(10)#send a SIGALRM x seconds from now
try:
    print("Before: %s" % time.strftime("%M:%S"))
    func_downsample_scatter_plot_AirGas(xlist_series=gasCV_vars_series,ylist_series=gasFdbk_vars_series)
except Exception as ex:
    if "FUBAR" in ex:
        print("Gotcha!")
    else:
        print("We're gonna need a bigger boat!")
else:
    print("It worked!")

In [ ]:
async def main():
    try:
        async with timeout(2*60):#2*60
            await (asyncio.gather(func_downsample_scatter_plotly_AirGas(xlist_series=gasCV_vars_series,ylist_series=gasFdbk_vars_series)))#sync_to_async()
    except: 
        print("The long operation timed out, but we've handled it.")
    else:
        print("It worked!")
    return 0

#asyncio.run(main())#For older versions
await main()

In [ ]:
async def main():
    try:
        async with timeout(2*60):#2*60
            await (asyncio.gather(func_downsample_scatter_plotly_AirGas(xlist_series=airCV_vars_series,ylist_series=airFdbk_vars_series)))#sync_to_async()
    except: 
        print("The long operation timed out, but we've handled it.")
    else:
        print("It worked!")
    return 0

#asyncio.run(main())#For older versions
await main()

In [ ]:
async def main():
    try:
        async with timeout(2*60):#2*60
            await (asyncio.gather(func_downsample_scatter_plotly_AirGas(xlist_series=airCV_vars_series,ylist_series=airSP_vars_series)))#sync_to_async()
    except: 
        print("The long operation timed out, but we've handled it.")
    else:
        print("It worked!")
    return 0

#asyncio.run(main())#For older versions
await main()

In [ ]:
async def main():
    try:
        async with timeout(2*60):#2*60
            await (asyncio.gather(func_downsample_scatter_plotly_AirGas(xlist_series=airFdbk_vars_series,ylist_series=airSP_vars_series)))#sync_to_async()
    except: 
        print("The long operation timed out, but we've handled it.")
    else:
        print("It worked!")
    return 0

#asyncio.run(main())#For older versions
await main()

In [ ]:
#Zone 0 Top: 5x5
zone0AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone0",regex=False)]
func_downsample_scatter_plot_zone(zone0AirGas_vars_series,"Zone0","Top","evenly")

x=dfZone0Scatter[var_list[3]].to_numpy()
y= dfZone0Scatter[var_list[1]].to_numpy()
regM = LinearRegression().fit(x.reshape(-1,1),y.reshape(-1,1))
y_pred = regM.predict(x.reshape(-1,1))
y_pred=y_pred[:,0]
#data=pd.merge(left=pd.DataFrame({"x":np.linspace(0,100,num=len(y)),"res":y-y_pred}),right=pd.DataFrame(SteelFamCluster,index=range(dfXY.shape[0]),columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)
data=pd.merge(left=pd.DataFrame({"x":x,"res":y-y_pred}),right=pd.DataFrame(SteelFamCluster,index=range(dfXY.shape[0]),columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True)
sns.scatterplot(data=data,x='x',y='res',hue='SteelFamCluster',style='SteelFamCluster',size=2)#,size=20
plt.scatter(x, y, color="green")
plt.plot(x, y_pred, color="blue", linewidth=3)
print(r2_score(y,y_pred))
print(mean_squared_error(y,y_pred))
#sns.scatterplot(pd.merge(left=dfZone0Scatter,right=pd.DataFrame(SteelFamCluster,index=dfXY.index,columns=['SteelFamCluster']),how='inner',right_index=True,left_index=True),x=var_list[3],y=var_list[0],hue='SteelFamCluster')#,size=20

In [ ]:
#Zone 0 Bottom: 5x5
zone0AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone0",regex=False)]
func_downsample_scatter_plot_zone(zone0AirGas_vars_series,"Zone0","Bottom","evenly")

In [ ]:
#Zone 1: 5x5
zone1AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone1",regex=False)]
func_downsample_scatter_plot_zone(zone1AirGas_vars_series,"Zone1","Top","evenly")

In [ ]:
#Zone 1: 5x5
zone1AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone1",regex=False)]
func_downsample_scatter_plot_zone(zone1AirGas_vars_series,"Zone1","Bottom","evenly")

In [ ]:
#Zone 2 Top: 5x5
zone2AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone2",regex=False)]
func_downsample_scatter_plot_zone(zone2AirGas_vars_series,"Zone2","Top","evenly")

In [ ]:
#Zone 2 Bottom: 5x5
zone2AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone2",regex=False)]
func_downsample_scatter_plot_zone(zone2AirGas_vars_series,"Zone2","Bottom","evenly")

In [ ]:
#Zone 3 Top: 5x5
zone3AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone3",regex=False)]
func_downsample_scatter_plot_zone(zone3AirGas_vars_series,"Zone3","Top","evenly")

In [ ]:
#Zone 3 Bottom: 5x5
zone3AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone3",regex=False)]
func_downsample_scatter_plot_zone(zone3AirGas_vars_series,"Zone3","Bottom","evenly")

In [ ]:
#Zone 4 Top: 5x5
zone4AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone4",regex=False)]
func_downsample_scatter_plot_zone(zone4AirGas_vars_series,"Zone4","Top","evenly")

In [ ]:
#Zone 4 Bottom: 5x5
zone4AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone4",regex=False)]
func_downsample_scatter_plot_zone(zone4AirGas_vars_series,"Zone4","Bottom","evenly")

In [ ]:
#Zone 5 Top: 5x5
zone5AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone5",regex=False)]
func_downsample_scatter_plot_zone(zone5AirGas_vars_series,"Zone5","Top","evenly")

In [ ]:
#Zone 5 Bottom: 5x5
zone5AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone5",regex=False)]
func_downsample_scatter_plot_zone(zone5AirGas_vars_series,"Zone5","Bottom","evenly")

In [ ]:
#Zone 6 Top: 5x5
zone6AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone6",regex=False)]
func_downsample_scatter_plot_zone(zone6AirGas_vars_series,"Zone6","Top","evenly")

In [ ]:
#Zone 6 Bottom: 5x5
zone6AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone6",regex=False)]
func_downsample_scatter_plot_zone(zone6AirGas_vars_series,"Zone6","Top","evenly")

In [ ]:
#Zone 7 Top: 5x5
zone7AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone7",regex=False)]
func_downsample_scatter_plot_zone(zone7AirGas_vars_series,"Zone7","Top","evenly")

In [ ]:
#Zone 7 Bottom: 5x5
zone7AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone7",regex=False)]
func_downsample_scatter_plot_zone(zone7AirGas_vars_series,"Zone7","Bottom","evenly")

In [ ]:
#Zone 8 Top: 5x5
zone8AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone8",regex=False)]
func_downsample_scatter_plot_zone(zone8AirGas_vars_series,"Zone8","Top","evenly")

In [ ]:
#Zone 8 Bottom: 5x5
zone8AirGas_vars_series=zoneAirGas_vars_series[zoneAirGas_vars_series.str.contains("Zone8",regex=False)]
func_downsample_scatter_plot_zone(zone8AirGas_vars_series,"Zone8","Bottom","evenly")

# Observation summary

Record high level comments about what you discovered